In [1]:
#import the neccessary libraries 
import csv
import pandas as pd
import requests
import json
from pprint import pprint 

# code necessary incase you'd like to search for a specific artist and their top 5 songs

artist = 'term=polyphia'
media = 'media=music'
entity = 'entity=song'
limit = 'limit=5'

url = 'http://itunes.apple.com/search?' + artist + media + entity + limit

In [2]:
#list of countries we narrow down to sort through our data

countries = ['hk', 'nl', 'ru', 'au', 'ca', 'fr', 'jp', 'gb', 'de', 'us' ]

In [3]:
#create a dataframe of the current top 5 songs in the US

#set up api url and output the response data

url = "https://rss.itunes.apple.com/api/v1/us/apple-music/top-songs/all/5/explicit.json"
response = requests.get(url).json()

#create lists that will take in the name, album, artist, genre, and rank of each song
song = []
album = []
artist = []
genre = []
rank = []

counter = 0

for i,t in enumerate (response["feed"]["results"]):
    name_1=(i,t["name"])
    album_1=(t["collectionName"])
    song.append(name_1[1])
    album.append(album_1)
    artist.append(t["artistName"])
    genre.append(t["genres"][0]["name"])
    rank.append(counter + 1)
    
    counter += 1

#use the data from above to create a dataframe of the top 5 songs
    
us = {
     "Name": song,
    "Album": album,
    "Artist": artist,
    "Genre" : genre,
    "Rank" : rank
}

us_df = pd.DataFrame(us)
us_df

,Name,Album,Artist,Genre,Rank
0,The Box,Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,1
1,Start wit Me (feat. Gunna),Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,2
2,Peta (feat. Meek Mill),Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,3
3,Tip Toe (feat. A Boogie wit da Hoodie),Please Excuse Me For Being Antisocial,Roddy Ricch,Hip-Hop/Rap,4
4,Woah,Woah - Single,Lil Baby,Hip-Hop/Rap,5


In [4]:
#create a dataframe similar to above that takes in all the top 5 songs from each country on our list

song_list = []
artist_list = []
album_list = []
country_list = []
genre_list = []
rank = []

for country in countries:
    top_list = []
    music_url = f"https://rss.itunes.apple.com/api/v1/{country}/apple-music/top-songs/all/5/explicit.json"
    
    try:
        response = requests.get(music_url).json()
        top_list.append(response["feed"]["results"])
        country_df = pd.DataFrame(top_list)
        for x in range (0,5):
            rank.append(x+1)
            song_list.append(country_df[x][0]['name'])
            country_list.append(country)
            artist_list.append(country_df[x][0]['artistName'])
            album_list.append(country_df[x][0]["collectionName"])
            genre_list.append(country_df[x][0]["genres"][0]["name"])
        
    except:
        print(f"Can't find {country}. Skipping... ")

top5_df = pd.DataFrame({
    "Country" : country_list,
    "Name" : song_list,
    "Artist" : artist_list,
    "Album" : album_list,
    "Genre" : genre_list,
    "Rank" : rank
})

top5_df['Country'] = top5_df['Country'].replace({'hk': 'Hong Kong',
                           'nl': 'Netherlands',
                           'ru': 'Russia',
                           'au': 'Australia',
                           'ca': 'Canada',
                           'fr': 'France',
                           'jp': 'Japan',
                           'gb': 'UK',
                           'de': 'Denmark',
                           'us': 'US'
                          })
top5_df['Source'] = 'Apple'
top5_df

,Country,Name,Artist,Album,Genre,Rank,Source
0,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,1,Apple
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,流行樂,2,Apple
2,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,國語流行樂,3,Apple
3,Hong Kong,Show Yourself,Idina Menzel & Evan Rachel Wood,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,4,Apple
4,Hong Kong,Memories,魔力紅樂團,Memories - Single,流行樂,5,Apple
5,Netherlands,Dance Monkey,Tones and I,Dance Monkey - Single,Alternative,1,Apple
6,Netherlands,All I Want For Christmas Is You,Mariah Carey,Merry Christmas,Feestdagen,2,Apple
7,Netherlands,Last Christmas (Single Version),Wham!,Last Christmas - Single,Feestdagen,3,Apple
8,Netherlands,Blinding Lights,The Weeknd,Blinding Lights - Single,R&B/Soul,4,Apple
9,Netherlands,everything i wanted,Billie Eilish,everything i wanted - Single,Alternative,5,Apple


In [6]:
from googletrans import Translator 
translator = Translator()

top5_df['Genre'] = top5_df['Genre'].apply(translator.translate, dest='en').apply(getattr, args=('text',))
top5_df['Genre'] = top5_df['Genre'].replace({'pop music' : 'Pop', 
                                                                   'Mandarin pop music' : 'Pop',
                                                                   'Alternative Music' : 'Alternative',
                                                                   'R & B, soul' : 'R&B/Soul',
                                                                   'priest' : 'Priest'
                                                                })
top5_df

,Country,Name,Artist,Album,Genre,Rank,Source
0,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,Soundtrack,1,Apple
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,Pop,2,Apple
2,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,Pop,3,Apple
3,Hong Kong,Show Yourself,Idina Menzel & Evan Rachel Wood,Frozen 2 (Original Motion Picture Soundtrack /...,Soundtrack,4,Apple
4,Hong Kong,Memories,魔力紅樂團,Memories - Single,Pop,5,Apple
5,Netherlands,Dance Monkey,Tones and I,Dance Monkey - Single,Alternative,1,Apple
6,Netherlands,All I Want For Christmas Is You,Mariah Carey,Merry Christmas,Holidays,2,Apple
7,Netherlands,Last Christmas (Single Version),Wham!,Last Christmas - Single,Holidays,3,Apple
8,Netherlands,Blinding Lights,The Weeknd,Blinding Lights - Single,R&B/Soul,4,Apple
9,Netherlands,everything i wanted,Billie Eilish,everything i wanted - Single,Alternative,5,Apple


In [7]:
new_file = top5_df.to_csv('../Resources/appleAPI_data.csv', index = None, header=True)

In [ ]:
#import spotify libraries
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

NADA_CLIENT_ID='435b32dd35304b11b6ab03d8463dd2bd'
NADA_CLIENT_SECRET='a86f35f553bf4b69a25ea28f6ea890b6'

In [ ]:
#get authentication from spotify's api 
spotify = spotipy.Spotify()
credentials = oauth2.SpotifyClientCredentials(
        client_id = NADA_CLIENT_ID,
        client_secret = NADA_CLIENT_SECRET)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

#extract Spotify ID from the top songs dataframe above , note spotify sometimes doesn't include features in the song name

spotify_id = []
for index, row in top5_df.iterrows():
    try:
        response = spotify.search(row["Name"], type="track", limit=1)
        spotify_id.append(response["tracks"]["items"][0]["album"]["artists"][0]["id"])
    except:
        text = row["Name"]
        head, sep, tail = text.partition('(')
        response = spotify.search(head, type="track", limit=1)
        spotify_id.append(response["tracks"]["items"][0]["album"]["artists"][0]["id"])
        continue
        
top5_df["Spotify ID"] = spotify_id
top5_df